This is the data cleaning and preprocessing for the data exploration project.

In [1]:
# Import modules and packages
import pandas as pd

!pip install openpyxl

In [2]:
# Import the dataset which requires cleaning
df_world = pd.read_csv("/content/drive/MyDrive/Monash_data_science/24s1/FIT5147/DEP/WDIData.csv")
# df_social = pd.read_csv("/content/drive/MyDrive/Monash_data_science/24s1/FIT5147/DEP/OECD.ELS.SPD,DSD_SOCX_AGG@DF_SOCX_AGG,1.0+.A..PT_B1GQ.ES10._T._T..csv")
df_leave = pd.read_csv("/content/drive/MyDrive/Monash_data_science/24s1/FIT5147/DEP/days-of-paid-maternity-paternity-and-parental-leave.csv")

<p style="text-align: center;">

</p>


### Dataset and wrangling process

This data exploration project used 3 datasets:

1. World Development Indicator
2. Social Expenditure
3. Days of leaves

We will inspect them one by one, and perform necessary steps to make them usable.

1. World Development Indicator Dataset

In [3]:
# Inspect the dataset to get basic ideas
df_world.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.392349,17.892005,18.359993,18.795151,19.295176,19.788156,20.279599,20.773627,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.720331,7.015917,7.281390,7.513673,7.809566,8.075889,8.366010,8.684137,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,38.184152,38.543180,38.801719,39.039014,39.323186,39.643848,39.894830,40.213891,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.859257,33.903515,38.851444,40.197332,43.028332,44.389773,46.268621,48.103609,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.623956,16.516633,24.594474,25.389297,27.041743,29.138285,30.998687,32.772690,NaN,NaN


In [4]:
# Check the dimensionality of the dataset
df_world.shape

(395276, 68)

In [5]:
# Drop a record when all cells are empty across the years
years = [str(year) for year in range(1960, 2023)]

df_world = df_world.dropna(subset=years, how="all")

# Drop columns that are entirely empty
df_world = df_world.dropna(how = 'all', axis=1)

In [6]:
# Drop "Indicator Code" and "Country Code" column because it's useless
df_world = df_world.drop(["Indicator Code"], axis=1)
df_world = df_world.drop(["Country Code"], axis=1)

In [7]:
# Check the dimensionality again
df_world.shape

(293591, 65)

In [8]:
indicator_list = df_world["Indicator Name"].unique().tolist()

# for item in indicator_list:
#   print(item)

df_indicator = pd.DataFrame(indicator_list)
df_indicator.to_csv("indicator.csv")

In [9]:
# check the number of features
len(indicator_list)

1486

In [10]:
country_list = df_world["Country Name"].unique().tolist()
len(country_list)

265

Given the size and complexity of this dataset:

(1) We are only performing the analysis on country level, instead of region or continent level, but kept the world data as benchmark for comparison. We remove country first to reduce computational expenses.

(2) We are only interested in features that are relevant to childbirth, gender, education and employment.

We will manually analysis the feature and perform the filtering and grouping.

In [11]:
country_to_remove = ['Africa Eastern and Southern',
 'Africa Western and Central',
 'Arab World',
 'Caribbean small states',
 'Central Europe and the Baltics',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'European Union',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Late-demographic dividend',
 'Latin America & Caribbean',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Least developed countries: UN classification',
 'Low & middle income',
 'Low income',
 'Lower middle income',
 'Middle East & North Africa',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Middle income',
 'North America',
 'Not classified',
 'OECD members',
 'Other small states',
 'Pacific island small states',
 'Post-demographic dividend',
 'Pre-demographic dividend',
 'Small states',
 'South Asia',
 'South Asia (IDA & IBRD)',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (excluding high income)',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Upper middle income']

df_world = df_world[~df_world["Country Name"].isin(country_to_remove)]

In [12]:
country_list_kept = df_world["Country Name"].unique().tolist()
len(country_list_kept)

218

After manually select interested features, the dataset has shown repeatition of features, espcially about school education. We will remove unnecessary features first, and then inspect the dataset to understand the reason behind those repeatation.

In [13]:
df_indicator = pd.read_excel("/content/drive/MyDrive/Monash_data_science/24s1/FIT5147/DEP/indicator_manual_selection.xlsx", sheet_name="all", header=None)

feature_to_keep = df_indicator[0].tolist()

In [14]:
df_world = df_world[df_world["Indicator Name"].isin(feature_to_keep)]

df_world = df_world.reset_index(drop=True)

In [15]:
df_world.to_csv("preview.csv", header=True, index=True)

In [16]:
df_features_world = pd.read_excel("/content/drive/MyDrive/Monash_data_science/24s1/FIT5147/DEP/indicator_manual_selection.xlsx", sheet_name="birth_world", header=None)

features_world = df_features_world[0].tolist()

df_world_birth = df_world[df_world["Indicator Name"].isin(features_world)]

In [17]:
df_world_birth = df_world_birth.melt(id_vars=['Country Name', 'Indicator Name'], var_name='Year', value_name='Value')

df_world_birth.to_csv("world_birth.csv")

In [18]:
df_world_birth

,Country Name,Indicator Name,Year,Value
0,World,"Adolescent fertility rate (births per 1,000 wo...",1960,91.748048
1,World,Births attended by skilled health staff (% of ...,1960,NaN
2,World,"Contraceptive prevalence, any modern method (%...",1960,NaN
3,World,"Fertility rate, total (births per woman)",1960,4.695854
4,World,Lifetime risk of maternal death (1 in: rate va...,1960,NaN
...,...,...,...,...
97897,Zimbabwe,"Fertility rate, total (births per woman)",2022,NaN
97898,Zimbabwe,Lifetime risk of maternal death (1 in: rate va...,2022,NaN
97899,Zimbabwe,Low-birthweight babies (% of births),2022,NaN
97900,Zimbabwe,Pregnant women receiving prenatal care (%),2022,NaN


In [19]:
df_china = df_world[df_world["Country Name"]=="China"]

In [20]:
# filter relevant dataset
df_china = df_world[df_world["Country Name"]=="China"]

# transform the format of dataset
df_china = df_china.drop(columns="Country Name").set_index("Indicator Name").transpose()

In [21]:
# filter the relevant year
df_china.index = df_china.index.astype(int)

df_china = df_china[df_china.index >= 1980]

df_china.index.astype(str)

Index(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [22]:
df_china.to_csv("china.csv")

In [23]:
df_china

Indicator Name,Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",Births attended by skilled health staff (% of total),"Contraceptive prevalence, any method (% of married women ages 15-49)","Contraceptive prevalence, any modern method (% of married women ages 15-49)",...,"Unemployment, total (% of total labor force) (national estimate)","Unemployment, youth female (% of female labor force ages 15-24) (national estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
1980,NaN,NaN,12.749,68.109636,7.379242,60.730393,18.21,NaN,NaN,54.6,...,4.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,16.332,65.525607,7.416471,58.109137,20.91,NaN,NaN,54.7,...,3.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,24.029,63.580339,7.463516,56.116823,22.28,NaN,NaN,58.2,...,3.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,NaN,NaN,25.563,61.376407,7.488467,53.887940,20.19,NaN,NaN,64.2,...,2.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,NaN,NaN,27.302,58.807946,7.501029,51.306917,19.90,NaN,NaN,66.3,...,1.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,NaN,NaN,28.228,56.578883,7.540722,49.038161,21.04,NaN,NaN,66.2,...,1.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986,NaN,NaN,28.796,54.793170,7.597524,47.195646,22.43,NaN,NaN,65.7,...,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,NaN,NaN,29.270,53.585668,7.664629,45.921040,23.33,NaN,NaN,69.3,...,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988,NaN,NaN,27.963,52.642336,7.751216,44.891120,22.37,NaN,NaN,73.2,...,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989,NaN,NaN,28.448,52.006354,7.882369,44.123985,21.58,NaN,NaN,NaN,...,2.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


------

2. Days of leaves (including paid maternity, paternity, and parental leave)

In [24]:
# Inspect the dataset
df_leave

,Entity,Code,Year,Length of paid maternity leave (calendar days),Length of paid parental leave for mother (calendar days),Length of paid paternity leave (calendar days),Length of paid parental leave for father (calendar days),Length of paid shared parental leave (calendar days)
0,Afghanistan,AFG,1970,0,0,0,0,0
1,Afghanistan,AFG,1971,0,0,0,0,0
2,Afghanistan,AFG,1972,0,0,0,0,0
3,Afghanistan,AFG,1973,0,0,0,0,0
4,Afghanistan,AFG,1974,0,0,0,0,0
...,...,...,...,...,...,...,...,...
10012,Zimbabwe,ZWE,2018,98,0,0,0,0
10013,Zimbabwe,ZWE,2019,98,0,0,0,0
10014,Zimbabwe,ZWE,2020,98,0,0,0,0
10015,Zimbabwe,ZWE,2021,98,0,0,0,0


In [25]:
df_leave = df_leave.drop(["Code"], axis=1)

df_leave

,Entity,Year,Length of paid maternity leave (calendar days),Length of paid parental leave for mother (calendar days),Length of paid paternity leave (calendar days),Length of paid parental leave for father (calendar days),Length of paid shared parental leave (calendar days)
0,Afghanistan,1970,0,0,0,0,0
1,Afghanistan,1971,0,0,0,0,0
2,Afghanistan,1972,0,0,0,0,0
3,Afghanistan,1973,0,0,0,0,0
4,Afghanistan,1974,0,0,0,0,0
...,...,...,...,...,...,...,...
10012,Zimbabwe,2018,98,0,0,0,0
10013,Zimbabwe,2019,98,0,0,0,0
10014,Zimbabwe,2020,98,0,0,0,0
10015,Zimbabwe,2021,98,0,0,0,0


In [26]:
df_leave = df_leave.melt(id_vars=['Entity', 'Year'], var_name='Leave Type', value_name='Days')

In [27]:
df_leave.to_csv("leave.csv")

In [28]:
df_leave

,Entity,Year,Leave Type,Days
0,Afghanistan,1970,Length of paid maternity leave (calendar days),0
1,Afghanistan,1971,Length of paid maternity leave (calendar days),0
2,Afghanistan,1972,Length of paid maternity leave (calendar days),0
3,Afghanistan,1973,Length of paid maternity leave (calendar days),0
4,Afghanistan,1974,Length of paid maternity leave (calendar days),0
...,...,...,...,...
50080,Zimbabwe,2018,Length of paid shared parental leave (calendar...,0
50081,Zimbabwe,2019,Length of paid shared parental leave (calendar...,0
50082,Zimbabwe,2020,Length of paid shared parental leave (calendar...,0
50083,Zimbabwe,2021,Length of paid shared parental leave (calendar...,0


### Data Checking

There are three dataframes used in the exploration project:
1. df_world_birth
2. df_china
3. df_leave

The steps we taken for data checking are similiar, and given the creditable source of those dataset, the dataframes are expected to be clean. Therefore, primary purpose of this data checking process is to validate those data are ready to use.

(1) Validate data types

In [29]:
df_world_birth.dtypes

Country Name       object
Indicator Name     object
Year               object
Value             float64
dtype: object

In [30]:
df_china.dtypes

Indicator Name
Adjusted net national income per capita (annual % growth)                            float64
Adjusted net national income per capita (constant 2015 US$)                          float64
Adolescent fertility rate (births per 1,000 women ages 15-19)                        float64
Age dependency ratio (% of working-age population)                                   float64
Age dependency ratio, old (% of working-age population)                              float64
                                                                                      ...   
Vulnerable employment, male (% of male employment) (modeled ILO estimate)            float64
Vulnerable employment, total (% of total employment) (modeled ILO estimate)          float64
Wage and salaried workers, female (% of female employment) (modeled ILO estimate)    float64
Wage and salaried workers, male (% of male employment) (modeled ILO estimate)        float64
Wage and salaried workers, total (% of total employment

In [31]:
df_leave.dtypes

Entity        object
Year           int64
Leave Type    object
Days           int64
dtype: object

(2) Range Check and looking for outliers

The range check are performed in google sheet by manually confirming with buildin summary statistics, and is performed on perview.csv and leave.csv.

(3) Missing value check

Both df_world_birth and df_china have large amount of missing value. However, after inspecting the raw data, I realised that those missing value are not missing at random, they are missing that depends on unobserved predictors, such as policy influence, social awareness ect. Therefore, to keep the relevant information, I made the decision to keep those missing values and study their pattern in the exploration project.

As for df_leave, there are no missing values in this dataset, and it is ready for use.

In [32]:
df_world_birth.isnull().sum()

Country Name          0
Indicator Name        0
Year                  0
Value             45610
dtype: int64

In [33]:
df_china.isnull().sum()

Indicator Name
Adjusted net national income per capita (annual % growth)                            17
Adjusted net national income per capita (constant 2015 US$)                          16
Adolescent fertility rate (births per 1,000 women ages 15-19)                         1
Age dependency ratio (% of working-age population)                                    0
Age dependency ratio, old (% of working-age population)                               0
                                                                                     ..
Vulnerable employment, male (% of male employment) (modeled ILO estimate)            11
Vulnerable employment, total (% of total employment) (modeled ILO estimate)          11
Wage and salaried workers, female (% of female employment) (modeled ILO estimate)    11
Wage and salaried workers, male (% of male employment) (modeled ILO estimate)        11
Wage and salaried workers, total (% of total employment) (modeled ILO estimate)      11
Length: 73, dtype

In [34]:
df_leave.isnull().sum()

Entity        0
Year          0
Leave Type    0
Days          0
dtype: int64

### Data Exploration Quick Checks

In [38]:
# Find the year where maternal dealth made it into world development indicator stats
df_maternal_death = df_world[df_world['Indicator Name']=="Lifetime risk of maternal death (1 in: rate varies by country)"]

non_empty_column = df_maternal_death.columns[df_maternal_death.notna().sum() > 0]

non_empty_column

Index(['Country Name', 'Indicator Name', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')